In [1]:
# *-* coding: utf-8 *-*
import itertools
from z3 import *

In [2]:
with open('gameboard.txt', 'r') as fin:
    circuit = fin.read()
    circuit = circuit.replace(' ', '0')
    circuit = [list(line) for line in circuit.split('\n') if len(line) > 0]

GRID_WIDTH = len(circuit[0])
GRID_HEIGHT = len(circuit)

In [3]:
# 3600 1632
print(GRID_WIDTH, GRID_HEIGHT)

3600 1632


In [4]:
s = Solver()
valList = [None] * GRID_WIDTH * GRID_HEIGHT
numList = []
formulas = []

for _, (x, y) in enumerate(itertools.product(range(GRID_WIDTH), range(GRID_HEIGHT))):
    value = int(circuit[y][x], 16)
    # 11 - fixed mine
    # 10 - chosen mine
    # 9 - closed
    if value == 11:
        valList[y * GRID_WIDTH + x] = IntVal(1)
    elif value == 9:
        val = Int(f"Cell{x},{y}")
        valList[y * GRID_WIDTH + x] = val
        formulas.append(val >= IntVal(0))
        formulas.append(val <= IntVal(1))
    elif value in range(1, 9):
        numList.append([x, y, value])
    elif value == 0:
        # empty cell
        pass
    else:
        print(f'Invalid state {value}')
        raise Exception("Invalid state")

In [5]:
for coord in numList:
    x, y, val = coord
    # if y > 0 and x > 0: neighbours += validate_grid[y-1][x-1].state in [10, 11]
    # if y > 0: neighbours += validate_grid[y-1][x].state in [10, 11]
    # if y > 0 and x+1 < GRID_WIDTH: neighbours += validate_grid[y-1][x+1].state in [10,11]
    # if x > 0: neighbours += validate_grid[y][x-1].state in [10, 11]
    # if x+1 < GRID_WIDTH: neighbours += validate_grid[y][x+1].state in [10,11]
    # if y+1 < GRID_HEIGHT and x > 0: neighbours += validate_grid[y+1][x-1].state in [10, 11]
    # if y+1 < GRID_HEIGHT: neighbours += validate_grid[y+1][x].state in [10, 11]
    # if y+1 < GRID_HEIGHT and x+1 < GRID_WIDTH: neighbours += validate_grid[y+1][x+1].state in [10, 11]
    formulas.append(IntVal(val) == 
          (valList[(y-1) * GRID_WIDTH + x-1] if y > 0 and x > 0 and valList[(y-1) * GRID_WIDTH + x-1] != None else IntVal(0)) +
          (valList[(y-1) * GRID_WIDTH + x] if y > 0 and valList[(y-1) * GRID_WIDTH + x] != None else IntVal(0)) +
          (valList[(y-1) * GRID_WIDTH + x+1] if y > 0 and x+1 < GRID_WIDTH and valList[(y-1) * GRID_WIDTH + x+1] != None else IntVal(0)) +
          (valList[y * GRID_WIDTH + x-1] if x > 0 and valList[y * GRID_WIDTH + x-1] != None else IntVal(0)) +
          (valList[y * GRID_WIDTH + x+1] if x+1 < GRID_WIDTH and valList[y * GRID_WIDTH + x+1] != None else IntVal(0)) +
          (valList[(y+1) * GRID_WIDTH + x-1] if y+1 < GRID_HEIGHT and x > 0 and valList[(y+1) * GRID_WIDTH + x-1] != None else IntVal(0)) +
          (valList[(y+1) * GRID_WIDTH + x] if y+1 < GRID_HEIGHT and valList[(y+1) * GRID_WIDTH + x] != None else IntVal(0)) +
          (valList[(y+1) * GRID_WIDTH + x+1] if y+1 < GRID_HEIGHT and x+1 < GRID_WIDTH and valList[(y+1) * GRID_WIDTH + x+1] != None else IntVal(0))
    )
print("constraints added finished")

constraints added finished


In [6]:
# https://stackoverflow.com/a/70656700/14646226 from alias by CC BY-SA 4.0
def all_smt(s, initial_terms):
    def block_term(s, m, t):
        s.add(t != m.eval(t, model_completion=True))
    def fix_term(s, m, t):
        s.add(t == m.eval(t, model_completion=True))
    def all_smt_rec(terms):
        if sat == s.check():
           m = s.model()
           yield m
           for i in range(len(terms)):
               s.push()
               block_term(s, m, terms[i])
               for j in range(i):
                   fix_term(s, m, terms[j])
               yield from all_smt_rec(terms[i:])
               s.pop()   
    yield from all_smt_rec(list(initial_terms))

In [7]:
# s = Solver()
# s.add(formulas)
# print(s.check())
# model = s.model()
ss = Solver()
ss.add(formulas)
models = list(all_smt(ss, [v for v in valList if v is IntVal]))

In [8]:
print("number of all possible models: ", len(models))

number of all possible models:  1


In [9]:
model = models[0]

In [10]:
bitsVariables = {}
for val in model:
    # any cell y=23
    if val.name().split(',')[1] == "23":
        # use x as key
        bitsVariables[int(val.name().split(',')[0].removeprefix("Cell"))] = model[val].as_long()
print(bitsVariables)
print(len(bitsVariables))

{3441: 0, 3417: 0, 3345: 0, 3321: 0, 3297: 0, 3225: 0, 3201: 0, 3177: 0, 3153: 1, 3081: 0, 3057: 0, 3033: 1, 3009: 0, 2985: 0, 2961: 0, 2937: 1, 2865: 0, 2841: 0, 2817: 0, 2793: 1, 2769: 0, 2745: 0, 2721: 1, 2697: 1, 2673: 0, 2649: 0, 2625: 0, 2553: 0, 2529: 0, 2505: 0, 2481: 1, 2457: 0, 2433: 0, 2409: 0, 2385: 1, 2361: 1, 2337: 1, 2313: 0, 2289: 1, 2217: 0, 2193: 0, 2169: 1, 2145: 0, 2121: 1, 2097: 1, 2073: 1, 2049: 0, 2025: 0, 2001: 0, 1977: 0, 1953: 1, 1929: 1, 1905: 0, 1833: 0, 1809: 0, 1785: 1, 1761: 0, 1737: 1, 1713: 1, 1689: 1, 1665: 1, 1641: 1, 1617: 1, 1593: 0, 1569: 0, 1545: 1, 1521: 0, 1497: 0, 1473: 0, 1449: 1, 1425: 1, 1401: 0, 1329: 0, 1305: 0, 1281: 0, 1257: 1, 1233: 0, 1209: 0, 1185: 0, 1161: 0, 1137: 0, 1113: 0, 1089: 0, 1065: 1, 1041: 0, 1017: 1, 993: 1, 969: 1, 945: 1, 921: 0, 897: 1, 873: 1, 849: 0, 825: 1, 801: 0, 777: 1, 705: 0, 681: 0, 657: 0, 633: 0, 609: 1, 585: 0, 561: 0, 537: 0, 513: 1, 489: 0, 465: 1, 441: 0, 417: 1, 393: 0, 369: 1, 345: 1, 321: 0, 297: 1, 2

In [11]:
bits = []
for x in range(GRID_WIDTH):
    if x in bitsVariables.keys():
        # close cell - can be mine or not
        bits.append(bitsVariables[x])
    elif int(circuit[23][x], 16) == 11:
        # fixed mine
        bits.append(1)
    elif valList[23 * GRID_WIDTH + x] == None and int(circuit[23][x], 16) in range(0, 9 ):
        # empty cell
        bits.append(0)
    else:
        print(f'Invalid state {value}')
        raise Exception("Invalid state")
print("".join([str(x) for x in bits]))
print("1s: ", len([x for x in bits if x == 1]))

0000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000001000000000000000000000000000000000000000000000001000000000000000000000001000000000000000000000001000000

In [12]:
import hashlib


flag = hashlib.sha256(bytes(bits)).hexdigest()
print(f'Flag: CTF{{{flag}}}')

Flag: CTF{d8675fca837faa20bc0f3a7ad10e9d2682fa0c35c40872938f1d45e5ed97ab27}
